In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cmdstanpy as stan
import os
np.set_printoptions(legacy="1.25")

df = pd.read_csv('siaya_CFP.csv')
df = df.assign(mort72 = df['hr72'] / df['total'])
df.head(3)

#downcasting happening automatically - will be deprecated in future python
repl = df['generation'].replace(         #replace generation with integer for STAN
    to_replace=df['generation'].unique(),
    value=range(1, 5)
    )


/home/daniel-hartman/anaconda3/envs/stan/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_13448/4177433217.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  repl = df['generation'].replace(         #replace generation with integer for STAN


In [7]:
mort_ind = []
for row in range(len(df)):
    mort_ind.extend(np.repeat(1, df['hr72'][row]))
    mort_ind.extend(np.repeat(0 , df['total'][row] - df['hr72'][row]))

replicate = []
for row in range(len(df)):
    replicate.extend(np.repeat(repl[row], df['total'][row]))

dose = []
for row in range(len(df)):
    dose.extend(np.repeat(df['Dose'][row], df['total'][row]))

logisticdata =  {
    "D": 1,
    "N": len(mort_ind),
    "L": 4,
    "y": mort_ind,
    "ll": replicate,
    "x": dose,
}
stan.write_stan_json("data.json", logisticdata)

In [8]:
#building the dictionary of data for STAN
bottle_data = {
    "D": 1,
    "N": len(mort_ind),
    "L": 4, #figure out how to get this automatically from unique values of 'generation'
    "y": mort_ind,
    "ll": replicate,
    "x": dose
}
stan.write_stan_json("data.json", bottle_data)

In [10]:
my_stanfile = os.path.join('.', 'logistic.stan')
my_model = stan.CmdStanModel(stan_file=my_stanfile, cpp_options={'STAN_THREADS':'true'})
my_model.name
my_model.stan_file
my_model.exe_file
my_model.code()
my_model.sample(data='data.json', show_console=True, iter_sampling=2000, 
                chains=1, output_dir=os.path.join('.', 'runfiles'), inits=0)

# bern_stanfile = os.path.join('.', 'bernoulli_simple.stan')
# bern_model = stan.CmdStanModel(stan_file=bern_stanfile, cpp_options={'STAN_THREADS':'true'})

# bern_model.sample(data='data.json', show_console=True, iter_sampling=2000, 
#                 chains=1, output_dir=os.path.join('.', 'runfiles'), inits=0)


14:45:52 - cmdstanpy - INFO - compiling stan file /home/daniel-hartman/Documents/OPEN_projects/Bayes_LC50/logistic.stan to exe file /home/daniel-hartman/Documents/OPEN_projects/Bayes_LC50/logistic
14:46:06 - cmdstanpy - INFO - compiled model executable: /home/daniel-hartman/Documents/OPEN_projects/Bayes_LC50/logistic
14:46:06 - cmdstanpy - INFO - Chain [1] start processing


Chain [1] method = sample (Default)
Chain [1] sample
Chain [1] num_samples = 2000
Chain [1] num_warmup = 1000 (Default)
Chain [1] save_warmup = false (Default)
Chain [1] thin = 1 (Default)
Chain [1] adapt
Chain [1] engaged = true (Default)
Chain [1] gamma = 0.05 (Default)
Chain [1] delta = 0.8 (Default)
Chain [1] kappa = 0.75 (Default)
Chain [1] t0 = 10 (Default)
Chain [1] init_buffer = 75 (Default)
Chain [1] term_buffer = 50 (Default)
Chain [1] window = 25 (Default)
Chain [1] save_metric = false (Default)
Chain [1] algorithm = hmc (Default)
Chain [1] hmc
Chain [1] engine = nuts (Default)
Chain [1] nuts
Chain [1] max_depth = 10 (Default)
Chain [1] metric = diag_e (Default)
Chain [1] metric_file =  (Default)
Chain [1] stepsize = 1 (Default)
Chain [1] stepsize_jitter = 0 (Default)
Chain [1] num_chains = 1 (Default)
Chain [1] id = 1 (Default)
Chain [1] data
Chain [1] file = data.json
Chain [1] init = 0
Chain [1] random
Chain [1] seed = 17271
Chain [1] output
Chain [1] file = /home/daniel-

14:46:10 - cmdstanpy - INFO - Chain [1] done processing
14:46:10 - cmdstanpy - WARNING - Some chains may have failed to converge.
	Chain 1 had 1 divergent transitions (0.1%)
	Use the "diagnose()" method on the CmdStanMCMC object to see further information.


Chain [1] Iteration: 2800 / 3000 [ 93%]  (Sampling)
Chain [1] Iteration: 2900 / 3000 [ 96%]  (Sampling)
Chain [1] Iteration: 3000 / 3000 [100%]  (Sampling)
Chain [1] 
Chain [1] Elapsed Time: 2.979 seconds (Warm-up)
Chain [1] 1.494 seconds (Sampling)
Chain [1] 4.473 seconds (Total)
Chain [1] 
Chain [1] 


CmdStanMCMC: model=logistic chains=1['method=sample', 'num_samples=2000', 'algorithm=hmc', 'adapt', 'engaged=1']
 csv_files:
	/home/daniel-hartman/Documents/OPEN_projects/Bayes_LC50/runfiles/logistic-20250510144606.csv
 output_files:
	/home/daniel-hartman/Documents/OPEN_projects/Bayes_LC50/runfiles/logistic-20250510144606_0-stdout.txt